In [4]:
import os
from time import sleep
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "chromedriver"
driver = webdriver.Chrome()  # chromedriver 열기


def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['제주 맛집']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)

    driver.quit()
    print("finish")
def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()
def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        if i >= 6:
            i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환
def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()



    


##### 0
#### 광고카페라라라
바다 앞이라 뷰가 너무 좋아요~ 당근케이크도 맛있고 당근 착즙주스랑 잘어울려요! 나이별 컵홀더를 주시는것도 재미있어요!/5
감성 카페? 뭐가 감성인지는 잘 모르겠으나 아무튼 요즘 말하는 전형적인 감성 카페임 ㅎ 카카오 맵 리뷰이벤트 하면 서비스가 있기때문에 이 막대한 리뷰작성 수를 보유하면서도 4점 후반대를 유지는게 아닐까?  솔직히 가격이 좀 비싸서 그렇지 뷰 좋고 맛도 어느정도 괜찮은 가게인건 맞으나 평점보고 눈 돌아가서 꼭 가야하는 곳으로 생각하는 오류는 범하지 마시길...../2
위치가 좋아요. 시원한 바다 바로 앞이라 음료 시키고 감상하기에 딱인 곳. 당근케익은 너무 달고 빵은 살짝 떡처럼 꾸덕해서 제 입맛엔 안맞았고 당근쥬스는 그냥 당근쥬스에요. 가격도 좀 있어요. /4
당근케이크 느끼하지 않고 맛있어요 !/5
작년에 오고 또 왔어요! 당근주스도 맛있고, 세화 앞바다 풍경도 예쁜 카페입니다 :)/5
에메랄드 바닷빛, 다양한 사진 스팟이 있어요!  구좌당근주스는 꼭 드셔보세요^^/5
넘 예쁜 세화해변의 바다를 바라보며 그색에 취하고,그기억을 옆서에 담아 보낸다는~~♡/4
카페 앞 바다가 너무 깨끗하고 예뻐서 뷰 맛집 당근주스는 100퍼 착즙이라서 그냥 인공적인 단맛이 아니고 당근의 단맛이 잘 느껴지는 맛이에요/5
당근 디저트도 맛있었고 엽서로 그림 그리면 집으로 보내주신다고 해서 잠시 몰입했네요. 뷰도 좋고 여러모로 힐링되었습니다/5
일단 뷰가 너무 좋고 당근이 엄청 크고 신선도도 느껴져요~~ 후후 /5
뷰가 너무 이뻐서 한참을 있다가 갑니다! 뷰 맛집 인정! 야외테이블에서 드세요!!/5
뷰도 좋고 케이크 맛있어용!!!! 고양이도 너무 귀여워요 한라봉차 맛있어요!/5
밤바다 보러 나왔다가 카페에 들어오게 되었는데, 분위기도, 맛도, 전망도 너무 좋네요ㅎㅎ 힐링!! 바다가 보이는 스팟마다 포토존 마련도 되어있어서  맑은 날 낮에 오면 더 좋을듯(๑˃̵ᴗ˂̵๑)/5
해변에서 경치 구경하며 즐기기 좋은 카페였습니다./5

TypeError: object of type 'NoneType' has no len()